In [ ]:
from openpyxl import load_workbook, Workbook

In [ ]:
class __excel:
    def __init__(self, name):
        self.i = 0
        self.name = name
        self.filename = self.name + ".xlsx"
        workbook = Workbook()
        workbook.save(filename=self.filename)
    def get_sheets(self):
        workbook = load_workbook(filename=self.filename)
        return workbook.sheetnames
    def add_columns(self, *col_name):
        workbook = load_workbook(filename=self.filename)
        for value in col_name:
            if value not in workbook.sheetnames:
                col_name = workbook.create_sheet(value, self.i)
                self.i += 1
            else:
                print(f"{value} already exists in {self.name}. Ignoring column creation!")
        if "Sheet" in workbook.sheetnames:
            del workbook["Sheet"]
        workbook.save(filename=self.filename)
    def write(self, sheet, key, index, value):
        workbook = load_workbook(filename=self.filename)
        ws = workbook[f"{sheet}"]
        ws[f"{key}{index}"] = value
        workbook.save(filename=self.filename)       

In [ ]:
book1 = __excel("Test")

In [ ]:
book1.add_columns("Sheet1","Sheet2","Sheet3")

In [ ]:
book1.get_sheets()

In [ ]:
info = ["192.168.0.1","192.168.0.2","192.168.0.3","192.168.0.4","192.168.0.5","192.168.0.6"]

In [ ]:
index = 1
for value in info:
    print(f"Adding {value} to Sheet at index A{index}")
    book1.write("Sheet1", "A" ,index , value)
    print(index)
    index += 1